## **Transformar datos para obtener un ranking organizado de los pilotos**

*   Trabajamos con el resultado obtenido en el notebook **race_results**

In [ ]:
# Creamos un parametro para la fecha del archivo
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
# Llamamos al notebook que contiene las variables de configuración
%run "../utils/configuration"

In [ ]:
# Llamamos al notebook que contiene funciones comunes
%run "../includes/common_functions"

### Paso 1 - Leer **race_results** de la capa **presentation**

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc
from pyspark.sql.functions import sum, when, count, col

In [ ]:
# El parámetro "presentation_folder_path" se encuentra en el notebook "configuration"
race_results_base = spark.read.parquet(f"{presentation_folder_path}/race_results")

In [ ]:
# Se filtra el dataframe "race_results" por el parametro "v_file_date"
race_results_list = race_results_base.filter(f"file_date = '{v_file_date}'")

In [ ]:
# La función "df_column_to_list()" se encuentra en el notebook "common_functions"
race_year_list = df_column_to_list(race_results_list, "race_year")

In [ ]:
race_results_df = race_results_base.filter(col("race_year").isin(race_year_list))

### Paso 2 - Aplicar una agrupación y agregación a **race_results**

In [ ]:
# Cuando posicion es igual a 1 (es decir, el piloto ha ganado la carrera), entonces, es igual a 1 (True = 1 y False = 0)
driver_standings_df = race_results_df \
.groupBy("race_year", "driver_name", "driver_nationality", "team") \
.agg(sum("points").alias("total_points"),
     count(when(col("position") == 1, True)).alias("wins")).sort(col('wins').desc()) 

### Paso 3 - Al resultado previo aplicar una ventana y calcular el ranking

In [ ]:
driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = driver_standings_df.withColumn("rank", rank().over(driver_rank_spec))

### Paso 4 - Escribir datos en el datalake como **parquet** y crear la tabla **driver_standings** en la base de datos **f1_presentation**

In [ ]:
# La función "overwrite_partition()" se encuentra en el notebook "common_functions"
# Recordar que esta función se llamará al utilizar %run "../includes/common_functions"
overwrite_partition(final_df, 'f1_presentation', 'driver_standings', 'race_year')